# python连接Oracle数据库

In [1]:
import pandas as pd
import cx_Oracle
import os
os.environ['path']=r'D:\QlSql\instantclient_11_2'  #设置instantclient环境变量位置
connection=cx_Oracle.connect('sys','linzch2020','202.38.228.21:1521/ORCL',cx_Oracle.SYSDBA)#需要以SYSDBA身份进行登录，否则会报错

In [3]:
# 获取每条高速公路的roadid
sql='select roadname,roadid from tollstation2021 group by roadid,roadname'
pd1=pd.read_sql(sql,connection)
roadinfo=dict(zip(pd1['ROADNAME'],pd1['ROADID']))
len(roadinfo)

168

# 以2021年6月份为例，计算整个月份，6种不同车型的平均值

In [11]:
day_list=['0'+str(x) for x in range(1,30) if x <10 ]#日期列表
day_list.extend([str(x) for x in range(10,31)])
id=roadinfo['广佛高速']#该路id
day='GDFTMJTX20210301'#表名，选择天数
per_weight={}
#获取当天每个车型的平均入口车重
for i in day_list:
    day='GDFTMJTX202106'+i
    print(day)
    s1='select vehicletype,avg(enweight) from '+ day + ' where vehicletype between 11 and 17 and enweight!=0 group by vehicletype'#查询每种车型的平均重量
    print(s1)
    pd2=pd.read_sql(s1,connection)
    if i=='01':
        print(i)
        per_weight=dict(zip(pd2['VEHICLETYPE'],pd2['AVG(ENWEIGHT)']))
        print('per_weight')
        print(per_weight)
    else:
        temp_dict=dict(zip(pd2['VEHICLETYPE'],pd2['AVG(ENWEIGHT)']))
        print('temp_dict')
        print(temp_dict)
        for j in range(11,17):
            print(j)
            per_weight[j]+=temp_dict[j]

GDFTMJTX20210601
select vehicletype,avg(enweight) from GDFTMJTX20210601 where vehicletype between 11 and 17 and enweight!=0 group by vehicletype
01
per_weight
{14: 29969.3513682281, 15: 32744.585888187557, 12: 16004.182580785426, 11: 10564.332840964456, 13: 32725.177756656707, 16: 42895.50924186458}
GDFTMJTX20210602
select vehicletype,avg(enweight) from GDFTMJTX20210602 where vehicletype between 11 and 17 and enweight!=0 group by vehicletype
temp_dict
{14: 31719.761939776425, 15: 33540.232922437266, 12: 16236.42133223468, 11: 10432.275557577344, 13: 31794.495876115758, 16: 44778.464340715356}
11
12
13
14
15
16
GDFTMJTX20210603
select vehicletype,avg(enweight) from GDFTMJTX20210603 where vehicletype between 11 and 17 and enweight!=0 group by vehicletype
temp_dict
{14: 31451.139782713326, 15: 35748.874783177875, 11: 10418.583282218233, 12: 16148.736466160335, 13: 31956.308640002746, 16: 45417.68242903393}
11
12
13
14
15
16
GDFTMJTX20210604
select vehicletype,avg(enweight) from GDFTMJTX20

In [12]:
per_weight

{14: 908865.1645568935,
 15: 1249266.9009258011,
 12: 482438.12972211075,
 11: 297730.0756167585,
 13: 947537.8158009782,
 16: 1350970.434819797}

In [13]:
#6月份30天的6种车型的总重值
total_weight={14: 908865.1645568935,
 15: 1249266.9009258011,
 12: 482438.12972211075,
 11: 297730.0756167585,
 13: 947537.8158009782,
 16: 1350970.434819797}

In [17]:
CarTypeWeight=[x/30 for x in list(total_weight.values())]
CarTypeWeight=dict(zip(list(total_weight.keys()),CarTypeWeight))
CarTypeWeight
#存下pandas数据
df=pd.DataFrame(pd.Series(CarTypeWeight),columns=['avg_weight'])
df=df.reset_index().rename(columns={'index':'type'})
with pd.ExcelWriter('avg_weight.xlsx') as writer:
    df.to_excel(writer,sheet_name='avg_weight',float_format='%.5f')

# 1. 计算每条高速路，单天货运量

In [94]:
# mj数据表保留字段为：VEHICLEPLATE , VEHICLETYPE ,MAX(ENWEIGHT)
day_list=['0'+str(x) for x in range(1,30) if x <10 ]#日期列表
day_list.extend([str(x) for x in range(10,31)])#补充完整日期
total_volume=dict(zip(list(roadinfo.keys()),[0]*len(roadinfo.keys())))#创建货运量字典，并初始化值为0
#id=roadinfo['广佛高速']#该路id
day='GDFTMJTX20210601'#表名，选择天数
self_weight_list=[2036,5728,10547,13565 ,15290 ,16298]#自重列表
self_avg_weight=[7887.97054 ,10353.16606 ,21037.34471 ,16730.38010 ,26352.57448 ,28733.88280]#平均总重列表
self_weight_dict={'VEHICLETYPE' : [x for x in range(11,17)],'self_weight':self_weight_list}#创建对应的字典
self_avg_weight_dict={'VEHICLETYPE':[x for x in range(11,17)],'avg_weight': self_avg_weight}
df2=pd.DataFrame(self_weight_dict)#转化为dataframe，用于后续合并
df3=pd.DataFrame(self_avg_weight_dict)
for road_id in list(roadinfo.keys()):
    id=roadinfo[road_id]
    s2='select vehicleplate,vehicletype,max(enweight) from ' + day + ' where substr(gantryno,1,11)=' + '\''+id+'\''+\
    ' and (vehicletype between 11 and 17 ) group by vehicleplate,vehicletype '
    df1=pd.read_sql(s2,connection)#每条道路每天的初始数据表
    #合并自重表，新建一个字段为Net_volume，Net_volume=max(enweight)-weight;合并平均值表，如果Net_volume<=0则用平均值代替，最后求和总的Net_volume
    df1['Net_volume']=0
    #新建一个自重表，并与df表merge
    df4=df1.merge(df2,on=['VEHICLETYPE'])
    df4=df4.merge(df3,on=['VEHICLETYPE'])
    df4['MAX(ENWEIGHT)']=df4['MAX(ENWEIGHT)'].astype(float)#转化Maxweight属性类型
    df4['Net_volume']=df4['MAX(ENWEIGHT)']-df4['self_weight']
    df5=df4[df4['Net_volume']<=0]
    df5['Net_volume']=df5['avg_weight']
    df6=df4[df4['Net_volume']>0]
    frames=[df5,df6]
    df7=pd.concat(frames)
    total_volume[road_id]=df7['Net_volume'].sum()

<ipython-input-94-f9ae9f1b1b71>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Net_volume']=df5['avg_weight']
<ipython-input-94-f9ae9f1b1b71>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Net_volume']=df5['avg_weight']
<ipython-input-94-f9ae9f1b1b71>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [97]:
total_volume

{'新台高速': 64217999.241679996,
 '梅河高速': 65891215.03481,
 '广惠高速': 517112698.99601996,
 '京珠北': 130827366.09221,
 '阳阳高速': 55273336.72958,
 '清平一期': 7253.0,
 '机荷东': 92402492.3699,
 '增从高速': 263402.42239,
 '广深高速': 894403829.79465,
 '惠盐深圳': 110968686.29448,
 '连英高速': 111010334.53331,
 '汕汾高速': 263504134.20883,
 '海湾大桥': 52371295.38099,
 '渝湛高速': 132306692.73085,
 '开阳高速': 239487829.30967999,
 '黄埔大桥': 1949494.8238600001,
 '龙大高速': 24881.34471,
 '盐坝高速': 0.0,
 '普惠高速': 43059804.137,
 '港珠澳大桥': 0.0,
 '湛徐支线': 115325773.99819,
 '河惠莞': 72544197.02289,
 '潮漳高速': 52512059.128690004,
 '潮惠高速': 447513937.20076,
 '惠盐惠州': 131701207.40349999,
 '广明高速': 263495070.63241,
 '从莞惠州': 699182.99687,
 '沿海阳江': 27457492.20976,
 '机场高速': 16334111.85387,
 '惠深沿海': 106548782.22112,
 '东新高速': 509356880.97326,
 '广乐高速': 556729843.71917,
 '水官高速': 36318082.0454,
 '深汕东': 86501722.15374,
 '韶赣高速': 287204325.08997,
 '虎门大桥': 1786741.9176999999,
 '江珠珠海段': 70425530.01792,
 '潮莞高速': 92078904.82808,
 '北环高速': 16510838.07349,
 '西线一期': 243598525.77069,
 

In [103]:
len([x for x in list(total_volume.values()) if x!=0.0])

157

In [104]:
#存下pandas数据
result=pd.DataFrame(pd.Series(total_volume),columns=['total_weight'])
result=result.reset_index().rename(columns={'index':'roadname'})
with pd.ExcelWriter('20210601_total_weight.xlsx') as writer:
    result.to_excel(writer,sheet_name='20210601_total_weight',float_format='%.5f')

# 2. 定位联结湖南，江西，福建，广西，海口方向的主要高速公路，货车流量高发路段

## 湖南（二广高速，清连高速，京港澳高速，仁新高速）

In [28]:
#计算每个高速路，各门架每天通过的货车频次，导出到表格中
road_list=['二广高速','清连高速','京港澳高速','仁新高速']
day_list=['0'+str(x) for x in range(1,30) if x <10 ]#日期列表
day_list.extend([str(x) for x in range(10,31)])#补充完整日期
hour_list=['0'+str(x) for x in range(0,24) if x <10 ]#小时列表
hour_list.extend([str(x) for x in range(10,24)])

for day in day_list:
    for hour in hour_list:
        table='highway_mjtx_44_202106'#表名，选择天数
        table=table+day+hour
        print(table)
        id=roadinfo['二广高速']
        #从中选出门架上行编号数据
        s_mj_up='select gantryhex,count(*) from '+ table + ' where substr(gantryno,1,11)=' + '\'' + id + '\''+ ' and direction=1 and vehicletype between 11 and 17 group by gantryhex '
        #下行数据
        s_mj_down='select gantryhex,count(*) from '+ table + ' where substr(gantryno,1,11)=' + '\'' + id + '\''+ ' and direction=2 and vehicletype between 11 and 17 group by gantryhex '
        if (day=='01'and  hour =='00') :
            pd_up=pd.read_sql(s_mj_up,connection)
            pd_down=pd.read_sql(s_mj_down,connection)
        else:
            temp_up=pd.read_sql(s_mj_up,connection)
            temp_down=pd.read_sql(s_mj_down,connection)
            pd_up['COUNT(*)']=pd_up['COUNT(*)']+temp_up['COUNT(*)']
            pd_down['COUNT(*)']=pd_down['COUNT(*)']+temp_up['COUNT(*)']
    break
with pd.ExcelWriter('通往湖南高速公路门架数据.xlsx') as writer:
    pd_up.to_excel(writer,sheet_name='上行',float_format='%.5f')
    pd_down.to_excel(writer,sheet_name='下行',float_format='%.5f')

highway_mjtx_44_2021060100
highway_mjtx_44_2021060101
highway_mjtx_44_2021060102
highway_mjtx_44_2021060103
highway_mjtx_44_2021060104
highway_mjtx_44_2021060105
highway_mjtx_44_2021060106
highway_mjtx_44_2021060107
highway_mjtx_44_2021060108
highway_mjtx_44_2021060109
highway_mjtx_44_2021060110
highway_mjtx_44_2021060111
highway_mjtx_44_2021060112
highway_mjtx_44_2021060113
highway_mjtx_44_2021060114
highway_mjtx_44_2021060115
highway_mjtx_44_2021060116
highway_mjtx_44_2021060117
highway_mjtx_44_2021060118
highway_mjtx_44_2021060119
highway_mjtx_44_2021060120
highway_mjtx_44_2021060121
highway_mjtx_44_2021060122
highway_mjtx_44_2021060123


In [72]:
# 封装成函数，对各联结地进行计算
def cal_for_hjfgh(road_list:list,province:str):
    day_list=['0'+str(x) for x in range(1,30) if x <8 ]#日期列表
    #day_list.extend([str(x) for x in range(10,31)])#补充完整日期
    hour_list=['0'+str(x) for x in range(0,24) if x <10 ]#小时列表
    hour_list.extend([str(x) for x in range(10,24)])
    writer=pd.ExcelWriter('通往'+ province +'高速公路门架数据.xlsx')
    for road in road_list:
        for day in day_list:
            for hour in hour_list:
                table='highway_mjtx_44_202106'#表名，选择天数
                table=table+day+hour
                print(table)
                id=roadinfo[road]
                #从中选出门架上行编号数据
                s_mj_up='select gantryhex,count(*) from '+ table + ' where substr(gantryno,1,11)=' + '\'' + id + '\''+ ' and direction=1 and vehicletype between 11 and 17 group by gantryhex '
                #下行数据
                s_mj_down='select gantryhex,count(*) from '+ table + ' where substr(gantryno,1,11)=' + '\'' + id + '\''+ ' and direction=2 and vehicletype between 11 and 17 group by gantryhex '
                if (day=='01'and  hour =='00') :
                    pd_up=pd.read_sql(s_mj_up,connection)
                    pd_down=pd.read_sql(s_mj_down,connection)
                    pd_up['COUNT(*)'].fillna(0,inplace=True)
                    pd_down['COUNT(*)'].fillna(0,inplace=True)
                else:
                    temp_up=pd.read_sql(s_mj_up,connection)
                    temp_down=pd.read_sql(s_mj_down,connection)
                    #把NAN值给代替为0
                    temp_up['COUNT(*)'].fillna(0,inplace=True)
                    temp_down['COUNT(*)'].fillna(0,inplace=True)
                    pd_up['COUNT(*)']=pd_up['COUNT(*)']+temp_up['COUNT(*)']
                    pd_down['COUNT(*)']=pd_down['COUNT(*)']+temp_down['COUNT(*)']
                #break
            #break
        pd_up.to_excel(writer,road+'上行',float_format='%.5f')
        pd_down.to_excel(writer,road+'下行',float_format='%.5f')
    writer.save()

In [67]:
hunan_list=['二广高速','清连高速','仁新高速']
jiangxi_list=['韶赣高速','大广高速','粤赣高速','龙紫高速','平兴高速']
fujian_list=['天汕高速','梅大高速','潮漳高速','汕汾高速']
guangxi_list=['云梧高速','广佛肇','包茂高速','渝湛高速']
haikou_list=['湛徐高速','湛徐支线']

In [74]:
cal_for_hjfgh(haikou_list,'海口')

highway_mjtx_44_2021060100
highway_mjtx_44_2021060101
highway_mjtx_44_2021060102
highway_mjtx_44_2021060103
highway_mjtx_44_2021060104
highway_mjtx_44_2021060105
highway_mjtx_44_2021060106
highway_mjtx_44_2021060107
highway_mjtx_44_2021060108
highway_mjtx_44_2021060109
highway_mjtx_44_2021060110
highway_mjtx_44_2021060111
highway_mjtx_44_2021060112
highway_mjtx_44_2021060113
highway_mjtx_44_2021060114
highway_mjtx_44_2021060115
highway_mjtx_44_2021060116
highway_mjtx_44_2021060117
highway_mjtx_44_2021060118
highway_mjtx_44_2021060119
highway_mjtx_44_2021060120
highway_mjtx_44_2021060121
highway_mjtx_44_2021060122
highway_mjtx_44_2021060123
highway_mjtx_44_2021060200
highway_mjtx_44_2021060201
highway_mjtx_44_2021060202
highway_mjtx_44_2021060203
highway_mjtx_44_2021060204
highway_mjtx_44_2021060205
highway_mjtx_44_2021060206
highway_mjtx_44_2021060207
highway_mjtx_44_2021060208
highway_mjtx_44_2021060209
highway_mjtx_44_2021060210
highway_mjtx_44_2021060211
highway_mjtx_44_2021060212
h

In [75]:
cal_for_hjfgh(guangxi_list,'广西')

highway_mjtx_44_2021060100
highway_mjtx_44_2021060101
highway_mjtx_44_2021060102
highway_mjtx_44_2021060103
highway_mjtx_44_2021060104
highway_mjtx_44_2021060105
highway_mjtx_44_2021060106
highway_mjtx_44_2021060107
highway_mjtx_44_2021060108
highway_mjtx_44_2021060109
highway_mjtx_44_2021060110
highway_mjtx_44_2021060111
highway_mjtx_44_2021060112
highway_mjtx_44_2021060113
highway_mjtx_44_2021060114
highway_mjtx_44_2021060115
highway_mjtx_44_2021060116
highway_mjtx_44_2021060117
highway_mjtx_44_2021060118
highway_mjtx_44_2021060119
highway_mjtx_44_2021060120
highway_mjtx_44_2021060121
highway_mjtx_44_2021060122
highway_mjtx_44_2021060123
highway_mjtx_44_2021060200
highway_mjtx_44_2021060201
highway_mjtx_44_2021060202
highway_mjtx_44_2021060203
highway_mjtx_44_2021060204
highway_mjtx_44_2021060205
highway_mjtx_44_2021060206
highway_mjtx_44_2021060207
highway_mjtx_44_2021060208
highway_mjtx_44_2021060209
highway_mjtx_44_2021060210
highway_mjtx_44_2021060211
highway_mjtx_44_2021060212
h

In [76]:
cal_for_hjfgh(hunan_list,'湖南')

highway_mjtx_44_2021060100
highway_mjtx_44_2021060101
highway_mjtx_44_2021060102
highway_mjtx_44_2021060103
highway_mjtx_44_2021060104
highway_mjtx_44_2021060105
highway_mjtx_44_2021060106
highway_mjtx_44_2021060107
highway_mjtx_44_2021060108
highway_mjtx_44_2021060109
highway_mjtx_44_2021060110
highway_mjtx_44_2021060111
highway_mjtx_44_2021060112
highway_mjtx_44_2021060113
highway_mjtx_44_2021060114
highway_mjtx_44_2021060115
highway_mjtx_44_2021060116
highway_mjtx_44_2021060117
highway_mjtx_44_2021060118
highway_mjtx_44_2021060119
highway_mjtx_44_2021060120
highway_mjtx_44_2021060121
highway_mjtx_44_2021060122
highway_mjtx_44_2021060123
highway_mjtx_44_2021060200
highway_mjtx_44_2021060201
highway_mjtx_44_2021060202
highway_mjtx_44_2021060203
highway_mjtx_44_2021060204
highway_mjtx_44_2021060205
highway_mjtx_44_2021060206
highway_mjtx_44_2021060207
highway_mjtx_44_2021060208
highway_mjtx_44_2021060209
highway_mjtx_44_2021060210
highway_mjtx_44_2021060211
highway_mjtx_44_2021060212
h

In [77]:
cal_for_hjfgh(jiangxi_list,'江西')

highway_mjtx_44_2021060100
highway_mjtx_44_2021060101
highway_mjtx_44_2021060102
highway_mjtx_44_2021060103
highway_mjtx_44_2021060104
highway_mjtx_44_2021060105
highway_mjtx_44_2021060106
highway_mjtx_44_2021060107
highway_mjtx_44_2021060108
highway_mjtx_44_2021060109
highway_mjtx_44_2021060110
highway_mjtx_44_2021060111
highway_mjtx_44_2021060112
highway_mjtx_44_2021060113
highway_mjtx_44_2021060114
highway_mjtx_44_2021060115
highway_mjtx_44_2021060116
highway_mjtx_44_2021060117
highway_mjtx_44_2021060118
highway_mjtx_44_2021060119
highway_mjtx_44_2021060120
highway_mjtx_44_2021060121
highway_mjtx_44_2021060122
highway_mjtx_44_2021060123
highway_mjtx_44_2021060200
highway_mjtx_44_2021060201
highway_mjtx_44_2021060202
highway_mjtx_44_2021060203
highway_mjtx_44_2021060204
highway_mjtx_44_2021060205
highway_mjtx_44_2021060206
highway_mjtx_44_2021060207
highway_mjtx_44_2021060208
highway_mjtx_44_2021060209
highway_mjtx_44_2021060210
highway_mjtx_44_2021060211
highway_mjtx_44_2021060212
h

In [78]:
cal_for_hjfgh(fujian_list,'福建')
cal_for_hjfgh(fujian_list,'福建')

highway_mjtx_44_2021060100
highway_mjtx_44_2021060101
highway_mjtx_44_2021060102
highway_mjtx_44_2021060103
highway_mjtx_44_2021060104
highway_mjtx_44_2021060105
highway_mjtx_44_2021060106
highway_mjtx_44_2021060107
highway_mjtx_44_2021060108
highway_mjtx_44_2021060109
highway_mjtx_44_2021060110
highway_mjtx_44_2021060111
highway_mjtx_44_2021060112
highway_mjtx_44_2021060113
highway_mjtx_44_2021060114
highway_mjtx_44_2021060115
highway_mjtx_44_2021060116
highway_mjtx_44_2021060117
highway_mjtx_44_2021060118
highway_mjtx_44_2021060119
highway_mjtx_44_2021060120
highway_mjtx_44_2021060121
highway_mjtx_44_2021060122
highway_mjtx_44_2021060123
highway_mjtx_44_2021060200
highway_mjtx_44_2021060201
highway_mjtx_44_2021060202
highway_mjtx_44_2021060203
highway_mjtx_44_2021060204
highway_mjtx_44_2021060205
highway_mjtx_44_2021060206
highway_mjtx_44_2021060207
highway_mjtx_44_2021060208
highway_mjtx_44_2021060209
highway_mjtx_44_2021060210
highway_mjtx_44_2021060211
highway_mjtx_44_2021060212
h

# 3. 统计物流园区的吸引度

In [ ]:
# 遍历六月份的gps数据，根据起始点经纬度，计算每个物流园区的到发量（由距离公式，如果到圆心距离小于3km，则属于从该点出发或到达该点）
s1='select startpx,startpy,endpx,endpy from GPS_OUTPUT_RAW_202106_T1 group by startpx,startpy,endpx,endpy'
car_SE=pd.read_sql(s1,connection)

In [ ]:
car_SE=car_SE[['STARTPX','STARTPY','ENDPX','ENDPY']].values
car_SE

array([[113.202451,  23.341009, 113.440784,  22.55662 ],
       [113.114595,  23.039306, 113.040098,  22.959141],
       [113.528264,  23.1292  , 113.522968,  23.055634],
       ...,
       [113.317904,  23.433116, 113.318424,  23.433166],
       [113.41789 ,  22.895571, 113.506613,  22.86398 ],
       [113.240886,  23.280304, 113.239764,  23.285535]])

In [ ]:
hub_info=pd.read_excel('广东省物流园区信息(坐标更新版).xlsx')
hub_info=hub_info[['经度','维度']]
hub_info=hub_info.values
type(hub_info)

numpy.ndarray

In [ ]:
o=np.zeros(380)
d=np.zeros(380)
type(o)

numpy.ndarray

In [ ]:
import pydistance #导入该py，wgs84坐标系下经纬度计算距离。
for i in range(car_SE.shape[0]):
    for j in range(hub_info.shape[0]):
        car_slng,car_slat=car_SE[i,0],car_SE[i,1]#获取每次出行记录的起始点经纬度
        car_elng,car_elat=car_SE[i,2],car_SE[i,3] #终点经纬度
        hub_lng,hub_lat=hub_info[j,0],hub_info[j,1] #物流园区经纬度
        s_distance=pydistance.get_distance_wgs84(car_slng,car_slat,hub_lng,hub_lat) #计算距离
        e_distance=pydistance.get_distance_wgs84(car_elng,car_elat,hub_lng,hub_lat)
        if(s_distance<=3000): #如果小于3000m，则计数
            o[j]+=1
        if(e_distance<=3000):
            d[j]+=1
        #print(i,j)

# 将计算结果从ndarry转化为dataframe

In [ ]:
def ndarry_to_df(nd:np.ndarray,s:str):
    df=pd.DataFrame(nd)
    df=df/30
    df.columns=[s]
    df[s]=df[s].astype(np.int64)
    return df

In [ ]:
hub_info=pd.read_excel('广东省物流园区信息(坐标更新版).xlsx')
df_o=ndarry_to_df(o,'货车出发频次')
hub_info['货车出发频次']=df_o['货车出发频次']
df_d=ndarry_to_df(d,'货车到达频次')
hub_info['货车到达频次']=df_d['货车到达频次']
writer=pd.ExcelWriter('广东省物流园区货车频次.xlsx')
hub_info.to_excel(writer)
writer.save()

In [ ]:
o.max(),o.sum()

(16946.0, 937324.0)

In [ ]:
o.sum(axis=0)

937324.0